### Creating tools from functions

In [1]:
from langchain.tools import tool
import numpy as np

@tool
def addition(numbers: list[int]) -> int:
    """
    Description: Adds multiple numbers.
    Accepts a list of integers.
    """
    return np.sum(numbers)

# Correct invocation
result = addition.invoke({"numbers": [1, 23, 4, 56,]})  
print("Result:", result)


Result: 84


### Creating tools from Runnables

In [2]:
from langchain.schema.runnable import RunnableLambda
def addition(numbers: list[int]) -> int:
    """
    Description: Adds multiple numbers.
    Accepts a list of integers.
    """
    return np.sum(numbers)

runnable_with_addition=RunnableLambda(addition)




runnable_with_addition.invoke([1, 23, 4, 56,])  

84

### Subclass BaseTool

In [5]:
from langchain_core.language_models import GenericFakeChatModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages(
    [("human", "Hello. Please respond in the style of {answer_style}.")]
)

# Placeholder LLM
llm = GenericFakeChatModel(messages=iter(["HI my name is Aditya."]))

chain = prompt | llm | StrOutputParser()

as_tool = chain.as_tool(
    name="Style responder", description="Description of when to use tool."
)
result = as_tool.invoke({'answer_style': 'casual'})
result

C:\Users\Dell\AppData\Local\Temp\ipykernel_1464\1094450617.py:15: LangChainBetaWarning: This API is in beta and may change in the future.
  as_tool = chain.as_tool(


'HI my name is Aditya.'

In [79]:
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.agents import AgentType

llm=ChatGroq(model_name="llama3-70b-8192")
import numpy as np
from langchain.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b."""
    return a * b

@tool
def query():
    """call this tool to handle other queries"""
    return "hello"

tools = [multiply]  

llm_bind_tools=llm.bind_tools(tools)

from langchain_core.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(
        content="do not answer by yourself. always go to **tools"
    ),
    HumanMessage(
        content= "waht is apple"
    )
]

print(llm_bind_tools.invoke(messages))



content='' additional_kwargs={'tool_calls': [{'id': 'call_etcb', 'function': {'arguments': '{"a":1,"b":1}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 933, 'total_tokens': 981, 'completion_time': 0.139780547, 'prompt_time': 0.059468326, 'queue_time': 0.020157193000000004, 'total_time': 0.199248873}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-47d83eda-1a84-4b30-8d5c-fdd8fffc5dcd-0' tool_calls=[{'name': 'multiply', 'args': {'a': 1, 'b': 1}, 'id': 'call_etcb', 'type': 'tool_call'}] usage_metadata={'input_tokens': 933, 'output_tokens': 48, 'total_tokens': 981}


In [13]:
print(llm_bind.invoke("add five,seven and six and also multi"))

content='' additional_kwargs={'tool_calls': [{'id': 'call_zz0x', 'function': {'arguments': '{"numbers":["5","7","6"]}', 'name': 'addition'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 1298, 'total_tokens': 1370, 'completion_time': 0.06, 'prompt_time': 0.252752553, 'queue_time': 0.022899117999999996, 'total_time': 0.312752553}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-c2deea12-9c85-4447-b828-08e5a5a78077-0' tool_calls=[{'name': 'addition', 'args': {'numbers': ['5', '7', '6']}, 'id': 'call_zz0x', 'type': 'tool_call'}] usage_metadata={'input_tokens': 1298, 'output_tokens': 72, 'total_tokens': 1370}


In [5]:
print(llm.invoke("add five,seven and six and also multiply").content)

Let's add 5, 7, and 6:

5 + 7 = 12
12 + 6 = 18

Now, let's multiply 5, 7, and 6:

5 × 7 = 35
35 × 6 = 210

So, the results are:

* The sum is 18
* The product is 210


## React Agent Implementaion Using Langchain

In [131]:
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain import hub
import re


model = ChatGroq(model_name="llama3-70b-8192")

In [145]:
@tool
def extract_numbers_from_text(text: str) -> str:
    """Extracts numbers from user input using LLM."""
    prompt = f"Extract and return only the numerical values from the following text as space-separated numbers: '{text}'. If words represent numbers, convert them."
    response = model.invoke(prompt)
    return response.content.strip()

In [146]:
@tool
def multiply_numbers(input_text: str) -> str:
    """Uses LLM to extract numbers and perform multiplication."""
    
    # Use LLM to extract numbers from the input
    extracted_numbers = extract_numbers_from_text(input_text)
    
    # Convert extracted numbers into a list
    try:
        numbers = list(map(int, extracted_numbers.split()))
        
        if len(numbers) < 2:
            return "Please provide at least two numbers for multiplication."

        # Handle "itself" case (e.g., "five times itself" → 5 * 5)
        if "itself" in input_text and len(numbers) == 1:
            numbers.append(numbers[0])

        # Perform multiplication
        result = 1
        for num in numbers:
            result *= num

        return f"The result of multiplying {' × '.join(map(str, numbers))} is {result}."
    
    except ValueError:
        return "Could not extract valid numbers. Please provide a clear multiplication request."


In [147]:
prompt = hub.pull("hwchase17/react")


tools = [multiply_numbers,extract_numbers_from_text]

agent = create_react_agent(model, tools, prompt)


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [150]:
response = agent_executor.invoke({"input": "multipy two in five times itself"})
print(response)




> Entering new AgentExecutor chain...
Thought: I need to extract the numbers from the input text to understand what operation to perform.

Action: extract_numbers_from_text
Action Input: "multipy two in five times itself"The numerical values from the text are:

2 5Thought: Now that I have the numbers, I need to understand the operation to perform. The phrase "in five times itself" suggests that 5 should be multiplied by itself.

Action: multiply_numbers
Action Input: "five times itself"Could not extract valid numbers. Please provide a clear multiplication request.Thought: I understand that the phrase "in five times itself" means 5 multiplied by itself, which is 5 * 5 = 25.

Action: multiply_numbers
Action Input: "5 * 5"Could not extract valid numbers. Please provide a clear multiplication request.Thought: I need to perform the multiplication manually.

Action: N/A
Action Input: N/AN/A is not a valid tool, try one of [multiply_numbers, extract_numbers_from_text].Thought: I need to per